In [1]:
from __future__ import print_function
import findspark
findspark.init()
findspark.find()
import pyspark
from pyspark.sql import SparkSession
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator


In [2]:
# spark a spakr session 

spark = SparkSession.builder.appName("Calssification Decission Tree").getOrCreate()


In [12]:
# download the dataset from https://archive.ics.uci.edu/ml/datasets/iris

# Read the data from CSV 

df = spark.read.csv("iris.data")

In [14]:
# print the Schmea 

df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)



In [18]:
#show data ( default 20 records )

df.show()

+---+---+---+---+-----------+
|_c0|_c1|_c2|_c3|        _c4|
+---+---+---+---+-----------+
|5.1|3.5|1.4|0.2|Iris-setosa|
|4.9|3.0|1.4|0.2|Iris-setosa|
|4.7|3.2|1.3|0.2|Iris-setosa|
|4.6|3.1|1.5|0.2|Iris-setosa|
|5.0|3.6|1.4|0.2|Iris-setosa|
|5.4|3.9|1.7|0.4|Iris-setosa|
|4.6|3.4|1.4|0.3|Iris-setosa|
|5.0|3.4|1.5|0.2|Iris-setosa|
|4.4|2.9|1.4|0.2|Iris-setosa|
|4.9|3.1|1.5|0.1|Iris-setosa|
|5.4|3.7|1.5|0.2|Iris-setosa|
|4.8|3.4|1.6|0.2|Iris-setosa|
|4.8|3.0|1.4|0.1|Iris-setosa|
|4.3|3.0|1.1|0.1|Iris-setosa|
|5.8|4.0|1.2|0.2|Iris-setosa|
|5.7|4.4|1.5|0.4|Iris-setosa|
|5.4|3.9|1.3|0.4|Iris-setosa|
|5.1|3.5|1.4|0.3|Iris-setosa|
|5.7|3.8|1.7|0.3|Iris-setosa|
|5.1|3.8|1.5|0.3|Iris-setosa|
+---+---+---+---+-----------+
only showing top 20 rows



In [25]:
# Lable indexing for Lables 

df.select( col("_c4")).distinct().show()

+---------------+
|            _c4|
+---------------+
| Iris-virginica|
|    Iris-setosa|
|Iris-versicolor|
+---------------+



In [30]:
stringIndexer = StringIndexer(inputCol="_c4", outputCol="label_index")
df_label_index = stringIndexer.fit(df).transform(df)

df_label_index.show()

+---+---+---+---+-----------+-----------+
|_c0|_c1|_c2|_c3|        _c4|label_index|
+---+---+---+---+-----------+-----------+
|5.1|3.5|1.4|0.2|Iris-setosa|        0.0|
|4.9|3.0|1.4|0.2|Iris-setosa|        0.0|
|4.7|3.2|1.3|0.2|Iris-setosa|        0.0|
|4.6|3.1|1.5|0.2|Iris-setosa|        0.0|
|5.0|3.6|1.4|0.2|Iris-setosa|        0.0|
|5.4|3.9|1.7|0.4|Iris-setosa|        0.0|
|4.6|3.4|1.4|0.3|Iris-setosa|        0.0|
|5.0|3.4|1.5|0.2|Iris-setosa|        0.0|
|4.4|2.9|1.4|0.2|Iris-setosa|        0.0|
|4.9|3.1|1.5|0.1|Iris-setosa|        0.0|
|5.4|3.7|1.5|0.2|Iris-setosa|        0.0|
|4.8|3.4|1.6|0.2|Iris-setosa|        0.0|
|4.8|3.0|1.4|0.1|Iris-setosa|        0.0|
|4.3|3.0|1.1|0.1|Iris-setosa|        0.0|
|5.8|4.0|1.2|0.2|Iris-setosa|        0.0|
|5.7|4.4|1.5|0.4|Iris-setosa|        0.0|
|5.4|3.9|1.3|0.4|Iris-setosa|        0.0|
|5.1|3.5|1.4|0.3|Iris-setosa|        0.0|
|5.7|3.8|1.7|0.3|Iris-setosa|        0.0|
|5.1|3.8|1.5|0.3|Iris-setosa|        0.0|
+---+---+---+---+-----------+-----

In [31]:
df_label_index.select( col("label_index")).distinct().show()

+-----------+
|label_index|
+-----------+
|        0.0|
|        1.0|
|        2.0|
+-----------+



In [32]:
df_label_index.show()

+---+---+---+---+-----------+-----------+
|_c0|_c1|_c2|_c3|        _c4|label_index|
+---+---+---+---+-----------+-----------+
|5.1|3.5|1.4|0.2|Iris-setosa|        0.0|
|4.9|3.0|1.4|0.2|Iris-setosa|        0.0|
|4.7|3.2|1.3|0.2|Iris-setosa|        0.0|
|4.6|3.1|1.5|0.2|Iris-setosa|        0.0|
|5.0|3.6|1.4|0.2|Iris-setosa|        0.0|
|5.4|3.9|1.7|0.4|Iris-setosa|        0.0|
|4.6|3.4|1.4|0.3|Iris-setosa|        0.0|
|5.0|3.4|1.5|0.2|Iris-setosa|        0.0|
|4.4|2.9|1.4|0.2|Iris-setosa|        0.0|
|4.9|3.1|1.5|0.1|Iris-setosa|        0.0|
|5.4|3.7|1.5|0.2|Iris-setosa|        0.0|
|4.8|3.4|1.6|0.2|Iris-setosa|        0.0|
|4.8|3.0|1.4|0.1|Iris-setosa|        0.0|
|4.3|3.0|1.1|0.1|Iris-setosa|        0.0|
|5.8|4.0|1.2|0.2|Iris-setosa|        0.0|
|5.7|4.4|1.5|0.4|Iris-setosa|        0.0|
|5.4|3.9|1.3|0.4|Iris-setosa|        0.0|
|5.1|3.5|1.4|0.3|Iris-setosa|        0.0|
|5.7|3.8|1.7|0.3|Iris-setosa|        0.0|
|5.1|3.8|1.5|0.3|Iris-setosa|        0.0|
+---+---+---+---+-----------+-----

In [34]:
df_label_index = df_label_index.drop("_c4")

df_label_index.show()

+---+---+---+---+-----------+
|_c0|_c1|_c2|_c3|label_index|
+---+---+---+---+-----------+
|5.1|3.5|1.4|0.2|        0.0|
|4.9|3.0|1.4|0.2|        0.0|
|4.7|3.2|1.3|0.2|        0.0|
|4.6|3.1|1.5|0.2|        0.0|
|5.0|3.6|1.4|0.2|        0.0|
|5.4|3.9|1.7|0.4|        0.0|
|4.6|3.4|1.4|0.3|        0.0|
|5.0|3.4|1.5|0.2|        0.0|
|4.4|2.9|1.4|0.2|        0.0|
|4.9|3.1|1.5|0.1|        0.0|
|5.4|3.7|1.5|0.2|        0.0|
|4.8|3.4|1.6|0.2|        0.0|
|4.8|3.0|1.4|0.1|        0.0|
|4.3|3.0|1.1|0.1|        0.0|
|5.8|4.0|1.2|0.2|        0.0|
|5.7|4.4|1.5|0.4|        0.0|
|5.4|3.9|1.3|0.4|        0.0|
|5.1|3.5|1.4|0.3|        0.0|
|5.7|3.8|1.7|0.3|        0.0|
|5.1|3.8|1.5|0.3|        0.0|
+---+---+---+---+-----------+
only showing top 20 rows



In [36]:
# check if any null records present

from pyspark.sql.functions import col, count, isnan, when
df_label_index.select([count(when(col(c).isNull(), c)).alias(c) for c in df_label_index.columns]).show()

+---+---+---+---+-----------+
|_c0|_c1|_c2|_c3|label_index|
+---+---+---+---+-----------+
|  0|  0|  0|  0|          0|
+---+---+---+---+-----------+



In [37]:
# print Schema of new DF

df_label_index.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- label_index: double (nullable = false)



In [41]:
# convert the data type from Stign to Float

from pyspark.sql.functions import col
df_label_index = df_label_index.select(*(col(c).cast("float").alias(c) for c in df_label_index.columns))

df_label_index.show()

df_label_index.printSchema()


+---+---+---+---+-----------+
|_c0|_c1|_c2|_c3|label_index|
+---+---+---+---+-----------+
|5.1|3.5|1.4|0.2|        0.0|
|4.9|3.0|1.4|0.2|        0.0|
|4.7|3.2|1.3|0.2|        0.0|
|4.6|3.1|1.5|0.2|        0.0|
|5.0|3.6|1.4|0.2|        0.0|
|5.4|3.9|1.7|0.4|        0.0|
|4.6|3.4|1.4|0.3|        0.0|
|5.0|3.4|1.5|0.2|        0.0|
|4.4|2.9|1.4|0.2|        0.0|
|4.9|3.1|1.5|0.1|        0.0|
|5.4|3.7|1.5|0.2|        0.0|
|4.8|3.4|1.6|0.2|        0.0|
|4.8|3.0|1.4|0.1|        0.0|
|4.3|3.0|1.1|0.1|        0.0|
|5.8|4.0|1.2|0.2|        0.0|
|5.7|4.4|1.5|0.4|        0.0|
|5.4|3.9|1.3|0.4|        0.0|
|5.1|3.5|1.4|0.3|        0.0|
|5.7|3.8|1.7|0.3|        0.0|
|5.1|3.8|1.5|0.3|        0.0|
+---+---+---+---+-----------+
only showing top 20 rows

root
 |-- _c0: float (nullable = true)
 |-- _c1: float (nullable = true)
 |-- _c2: float (nullable = true)
 |-- _c3: float (nullable = true)
 |-- label_index: float (nullable = false)



In [62]:
features_cols =df_label_index.drop("label_index").columns

#Prepare features to proviude the the input for model 

assembler = VectorAssembler(inputCols=features_cols,outputCol="features")
df_model=assembler.transform(df_label_index)


#df_model.drop(features_cols.tolist())

df_model.show()


+---+---+---+---+-----------+--------------------+
|_c0|_c1|_c2|_c3|label_index|            features|
+---+---+---+---+-----------+--------------------+
|5.1|3.5|1.4|0.2|        0.0|[5.09999990463256...|
|4.9|3.0|1.4|0.2|        0.0|[4.90000009536743...|
|4.7|3.2|1.3|0.2|        0.0|[4.69999980926513...|
|4.6|3.1|1.5|0.2|        0.0|[4.59999990463256...|
|5.0|3.6|1.4|0.2|        0.0|[5.0,3.5999999046...|
|5.4|3.9|1.7|0.4|        0.0|[5.40000009536743...|
|4.6|3.4|1.4|0.3|        0.0|[4.59999990463256...|
|5.0|3.4|1.5|0.2|        0.0|[5.0,3.4000000953...|
|4.4|2.9|1.4|0.2|        0.0|[4.40000009536743...|
|4.9|3.1|1.5|0.1|        0.0|[4.90000009536743...|
|5.4|3.7|1.5|0.2|        0.0|[5.40000009536743...|
|4.8|3.4|1.6|0.2|        0.0|[4.80000019073486...|
|4.8|3.0|1.4|0.1|        0.0|[4.80000019073486...|
|4.3|3.0|1.1|0.1|        0.0|[4.30000019073486...|
|5.8|4.0|1.2|0.2|        0.0|[5.80000019073486...|
|5.7|4.4|1.5|0.4|        0.0|[5.69999980926513...|
|5.4|3.9|1.3|0.4|        0.0|[5

In [71]:
(train, test) = df_model.randomSplit([0.7, 0.3])


print("total _records" ,df_model.count())
print("train", train.count())
print("test", test.count())



total _records 150
train 102
test 48
150
+---+---+---+---+-----------+--------------------+
|_c0|_c1|_c2|_c3|label_index|            features|
+---+---+---+---+-----------+--------------------+
|4.3|3.0|1.1|0.1|        0.0|[4.30000019073486...|
|4.4|2.9|1.4|0.2|        0.0|[4.40000009536743...|
|4.4|3.2|1.3|0.2|        0.0|[4.40000009536743...|
|4.5|2.3|1.3|0.3|        0.0|[4.5,2.2999999523...|
|4.6|3.1|1.5|0.2|        0.0|[4.59999990463256...|
|4.6|3.2|1.4|0.2|        0.0|[4.59999990463256...|
|4.6|3.4|1.4|0.3|        0.0|[4.59999990463256...|
|4.6|3.6|1.0|0.2|        0.0|[4.59999990463256...|
|4.7|3.2|1.3|0.2|        0.0|[4.69999980926513...|
|4.8|3.0|1.4|0.1|        0.0|[4.80000019073486...|
|4.8|3.0|1.4|0.3|        0.0|[4.80000019073486...|
|4.8|3.1|1.6|0.2|        0.0|[4.80000019073486...|
|4.8|3.4|1.6|0.2|        0.0|[4.80000019073486...|
|4.8|3.4|1.9|0.2|        0.0|[4.80000019073486...|
|4.9|2.4|3.3|1.0|        1.0|[4.90000009536743...|
|4.9|2.5|4.5|1.7|        2.0|[4.900000095

In [72]:
#Sample tarin & test data
train.show()
test.show()

+---+---+---+---+-----------+--------------------+
|_c0|_c1|_c2|_c3|label_index|            features|
+---+---+---+---+-----------+--------------------+
|4.3|3.0|1.1|0.1|        0.0|[4.30000019073486...|
|4.4|2.9|1.4|0.2|        0.0|[4.40000009536743...|
|4.4|3.2|1.3|0.2|        0.0|[4.40000009536743...|
|4.5|2.3|1.3|0.3|        0.0|[4.5,2.2999999523...|
|4.6|3.1|1.5|0.2|        0.0|[4.59999990463256...|
|4.6|3.2|1.4|0.2|        0.0|[4.59999990463256...|
|4.6|3.4|1.4|0.3|        0.0|[4.59999990463256...|
|4.6|3.6|1.0|0.2|        0.0|[4.59999990463256...|
|4.7|3.2|1.3|0.2|        0.0|[4.69999980926513...|
|4.8|3.0|1.4|0.1|        0.0|[4.80000019073486...|
|4.8|3.0|1.4|0.3|        0.0|[4.80000019073486...|
|4.8|3.1|1.6|0.2|        0.0|[4.80000019073486...|
|4.8|3.4|1.6|0.2|        0.0|[4.80000019073486...|
|4.8|3.4|1.9|0.2|        0.0|[4.80000019073486...|
|4.9|2.4|3.3|1.0|        1.0|[4.90000009536743...|
|4.9|2.5|4.5|1.7|        2.0|[4.90000009536743...|
|4.9|3.0|1.4|0.2|        0.0|[4

In [76]:
# Decision Tree model initialisation and Training 

dtmodel = DecisionTreeClassifier(labelCol="label_index", featuresCol="features")

model =dtmodel.fit(train)





In [77]:
predictions = model.transform(test)

In [78]:
predictions.show()

+---+---+---+---+-----------+--------------------+--------------+-------------+----------+
|_c0|_c1|_c2|_c3|label_index|            features| rawPrediction|  probability|prediction|
+---+---+---+---+-----------+--------------------+--------------+-------------+----------+
|4.4|3.0|1.3|0.2|        0.0|[4.40000009536743...|[32.0,0.0,0.0]|[1.0,0.0,0.0]|       0.0|
|4.7|3.2|1.6|0.2|        0.0|[4.69999980926513...|[32.0,0.0,0.0]|[1.0,0.0,0.0]|       0.0|
|4.9|3.1|1.5|0.1|        0.0|[4.90000009536743...|[32.0,0.0,0.0]|[1.0,0.0,0.0]|       0.0|
|5.0|3.0|1.6|0.2|        0.0|[5.0,3.0,1.600000...|[32.0,0.0,0.0]|[1.0,0.0,0.0]|       0.0|
|5.0|3.2|1.2|0.2|        0.0|[5.0,3.2000000476...|[32.0,0.0,0.0]|[1.0,0.0,0.0]|       0.0|
|5.0|3.4|1.5|0.2|        0.0|[5.0,3.4000000953...|[32.0,0.0,0.0]|[1.0,0.0,0.0]|       0.0|
|5.0|3.5|1.3|0.3|        0.0|[5.0,3.5,1.299999...|[32.0,0.0,0.0]|[1.0,0.0,0.0]|       0.0|
|5.1|2.5|3.0|1.1|        1.0|[5.09999990463256...|[0.0,34.0,0.0]|[0.0,1.0,0.0]|       1.0|

In [84]:
evaluator = MulticlassClassificationEvaluator(labelCol="label_index", predictionCol="prediction", metricName="accuracy")

accuracy = evaluator.evaluate(predictions)

print( "Accuracy for DT model is " , round(accuracy*100,2),'%')


Accuracy for DT model is  95.83 %
